# Predictions of Druggable peptides using the best model

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# remove warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix,accuracy_score, roc_auc_score,f1_score, recall_score, precision_score
from sklearn.utils import class_weight

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import LinearSVC

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.feature_selection import RFECV, VarianceThreshold, SelectKBest, chi2
from sklearn.feature_selection import SelectFromModel, SelectPercentile, f_classif

import seaborn as sns; sns.set() # data visualization library 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from imblearn.over_sampling import SMOTE

from sklearn.datasets import load_iris
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, StratifiedKFold
import numpy as np

import pandas as pd
from sklearn.utils import class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.externals import joblib

print(__doc__)

In [ ]:
def DataCheckings(df):
    # CHECKINGS ***************************
    # Check the number of data points in the data set
    print("\nData points =", len(df))
    
    # Check the number of columns in the data set
    print("\nColumns (output + features)=",len(df.columns))
    
    # Check the data types
    print("\nData types =", df.dtypes.unique())
    
    # Dataset statistics
    print('\n')
    df.describe()
    
    # print names of columns
    print('Column Names:\n', df.columns)
    
    # see if there are categorical data
    print("\nCategorical features:", df.select_dtypes(include=['O']).columns.tolist())
    
    # Check NA values
    # Check any number of columns with NaN
    print("\nColumns with NaN: ", df.isnull().any().sum(), ' / ', len(df.columns))

    # Check any number of data points with NaN
    print("\nNo of data points with NaN:", df.isnull().any(axis=1).sum(), ' / ', len(df))

In [ ]:
def getDataFromDataset(sFile, OutVar):
    # read details file
    print('\n-> Read dataset', sFile)
    df = pd.read_csv(sFile)
    #df = feather.read_dataframe(sFile)
    
    DataCheckings(df)
    
    # remove duplicates!
    df.drop_duplicates(keep=False, inplace=True)
    
    print('Shape', df.shape)
    # print(list(df.columns))

    # select X and Y
    ds_y = df[OutVar]
    ds_X = df.drop(OutVar,axis = 1)
    Xdata = ds_X.values # get values of features
    Ydata = ds_y.values # get output values

    print('Shape X data:', Xdata.shape)
    print('Shape Y data:',Ydata.shape)
    
    # return data for X and Y, feature names as list
    return (Xdata, Ydata, list(ds_X.columns))

In [ ]:
def  set_weights(y_data, option='balanced'):
    """Estimate class weights for umbalanced dataset
       If ‘balanced’, class weights will be given by n_samples / (n_classes * np.bincount(y)). 
       If a dictionary is given, keys are classes and values are corresponding class weights. 
       If None is given, the class weights will be uniform """
    cw = class_weight.compute_class_weight(option, np.unique(y_data), y_data)
    w = {i:j for i,j in zip(np.unique(y_data), cw)}
    return w 

In [ ]:
# define output variables
outVar = 'Class'

# define list of folds
foldType = 3

# define a label for output files
targetName = 'GS_Outer'

seed = 28

## Reproduce the pipeline model

In [ ]:
sFile = './datasets/ds.Class_TC_ballanced.csv'

# get data from file
Xdata, Ydata, Features = getDataFromDataset(sFile,outVar) # n_sample=100

In [ ]:
# Calculate class weights
class_weights = set_weights(Ydata)
print("Class weights = ", class_weights)

In [ ]:
outer_cv = StratifiedKFold(n_splits=3,shuffle=True,random_state=seed)

In [ ]:
ifold = 0
ACCs  =[]
AUROCs=[]
models =[]
SelectedFeatures =[]

for train_index, test_index in outer_cv.split(Xdata, Ydata):
    ifold +=1
    
    print("Fold =",ifold)
    start = time.time()
    
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = Xdata[train_index], Xdata[test_index]
    y_train, y_test = Ydata[train_index], Ydata[test_index]
    
    # Standardize dataset
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test  = scaler.transform(X_test)
    
    # Feature selection # FS = SelectFromModel(LinearSVC(), max_features = 400,threshold=-np.inf)
    lsvc = LinearSVC(max_iter=50000).fit(X_train, y_train)
    model = SelectFromModel(lsvc, prefit=True,max_features = 200,threshold=-np.inf)
    X_train = model.transform(X_train)
    X_test  = model.transform(X_test)
    #print("Selected X:", X_train.shape)

    # Selected features
    SelFeatures = []
    for i in model.get_support(indices=True):
        SelFeatures.append(Features[i])
    SelectedFeatures.append(SelFeatures)

    #scaler.transform(X_test)
    clf = SVC(kernel = 'rbf', random_state=seed,gamma='scale',
              class_weight=class_weights,probability=True)
    clf.fit(X_train, y_train)
    
    joblib.dump(clf, 'SVM_model'+str(ifold)+'.pkl', compress = 1)
    models.append(clf)
    
    y_pred = clf.predict_proba(X_test)
    AUROC = roc_auc_score(y_test, y_pred[:, 1])
    AUROCs.append(AUROC)
    
    ACC = clf.score(X_test,y_test)
    ACCs.append(ACC)
   
    print("AUROC=",AUROC,"ACC=",ACC, (time.time() - start)/60,"mins")
    

Let's see the mean AUROC values for best model and the standard deviations:

In [ ]:
print(np.mean(AUROCs),np.std(AUROCs))

In [ ]:
print(np.mean(ACCs),np.std(ACCs))

In [ ]:
# all the selected features for the 3 folds
print(SelectedFeatures)

In [ ]:
# differences of selected descriptors
print(list(set(SelectedFeatures[1])-set(SelectedFeatures[0])))

In [ ]:
# differences of selected descriptors
print(list(set(SelectedFeatures[1])-set(SelectedFeatures[2])))

## Predictions with the best model

We choose model 2 as the best due to the maximum ACC value (AUROC= 0.9752, ACC= 0.937).

In [ ]:
# the selected features for model 2
print(SelectedFeatures[1])

Load the prediction datasets (the same format as the dataset: 8000 TC features + Class=-1):

In [ ]:
# get data from files and check the files
sFile1 = './datasets/ds.Screening_1_TC.csv'
Xdata1, Ydata1, Features1 = getDataFromDataset(sFile1,outVar) 

sFile2 = './datasets/ds.Screening_2_TC.csv'
Xdata2, Ydata2, Features2 = getDataFromDataset(sFile2,outVar) 

sFile3 = './datasets/ds.Screening_3_TC.csv'
Xdata3, Ydata3, Features3 = getDataFromDataset(sFile3,outVar)

print(Xdata2.shape,Xdata3.shape)

Use only the second split / model - scale the prediction datasets, select only the features of model 2, predict the class and predict the probability of that class:

In [ ]:
ifold = 0

for train_index, test_index in outer_cv.split(Xdata, Ydata):
    ifold +=1
    
    if ifold ==2: # only model 2
        print("Fold =",ifold)
        start = time.time()

        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = Xdata[train_index], Xdata[test_index]
        y_train, y_test = Ydata[train_index], Ydata[test_index]

        # Standardize dataset
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test  = scaler.transform(X_test)
        # scale prediction set
        Xdata1  = scaler.transform(Xdata1)
        Xdata2  = scaler.transform(Xdata2)
        Xdata3  = scaler.transform(Xdata3)
        
        # Feature selection # FS = SelectFromModel(LinearSVC(), max_features = 400,threshold=-np.inf)
        lsvc = LinearSVC(max_iter=50000).fit(X_train, y_train)
        model = SelectFromModel(lsvc, prefit=True,max_features = 200,threshold=-np.inf)
        X_train = model.transform(X_train)
        X_test  = model.transform(X_test)
    
        # Selected features
        SelFeatures = []
        for i in model.get_support(indices=True):
            SelFeatures.append(Features[i])
        
        # apply selected features to prediction set
        Xdata1 = Xdata1[:, model.get_support()]
        print("Xdata1 sel=", Xdata1.shape)
        Xdata2 = Xdata2[:, model.get_support()]
        print("Xdata2 sel=", Xdata2.shape)
        Xdata3 = Xdata3[:, model.get_support()]
        print("Xdata3 sel=", Xdata3.shape)

        # we dont need to calculate again, but load the model from the disk!
        #clf = SVC(kernel = 'rbf', random_state=seed,gamma='scale',
        #          class_weight=class_weights,probability=True)
        #clf.fit(X_train, y_train)

        # load the saved model from disk
        clf = joblib.load('SVM_model'+str(ifold)+'.pkl')
        #joblib.dump(clf, 'SVM_model'+str(ifold)+'.pkl', compress = 1)
        
        # predictions with the model
        Ydata1 = clf.predict(Xdata1)
        Ydata2 = clf.predict(Xdata2)
        Ydata3 = clf.predict(Xdata3)
        
        # add probabilities (n_samples X n_classes; class 0, class 1)
        Ydata1prob = clf.predict_proba(Xdata1)
        Ydata2prob = clf.predict_proba(Xdata2)
        Ydata3prob = clf.predict_proba(Xdata3)
        
        # save predictions for list 1
        dff1 = pd.DataFrame(Xdata1,columns=SelFeatures)
        dff1['Class'] = Ydata1
        dff1['Prob0'] = Ydata1prob[:,0]
        dff1['Prob1'] = Ydata1prob[:,1]
        # merge with protein information from other file
        result = pd.concat([dff1, pd.read_csv('./PREDICTIONS/TC_seqs.Screening_1_Cancer_Driver_Genes.csv')], axis=1)
        # creat new order of columns in final results
        newHeader=['Class','Prob1','Prob0','V1','V2']
        result = result[newHeader]
        result = result.sort_values(by=['Prob1'], ascending=False)
        result.to_csv(sFile1[:-4]+'_predictions.csv', index=True)

        # save predictions for list 2
        dff2 = pd.DataFrame(Xdata2,columns=SelFeatures)
        dff2['Class'] = Ydata2
        dff2['Prob0'] = Ydata2prob[:,0]
        dff2['Prob1'] = Ydata2prob[:,1]
        # merge with protein information from other file
        result = pd.concat([dff2, pd.read_csv('./PREDICTIONS/TC_seqs.Screening_2_OncoOmics_Genes.csv')], axis=1)
        # creat new order of columns in final results
        newHeader=['Class','Prob1','Prob0','V1','V2']
        result = result[newHeader]
        result = result.sort_values(by=['Prob1'], ascending=False)
        result.to_csv(sFile2[:-4]+'_predictions.csv', index=True)
        
        # save predictions for list 3
        dff3 = pd.DataFrame(Xdata3,columns=SelFeatures)
        dff3['Class'] = Ydata3
        dff3['Prob0'] = Ydata3prob[:,0]
        dff3['Prob1'] = Ydata3prob[:,1]
        # merge with protein information from other file
        result = pd.concat([dff3, pd.read_csv('./PREDICTIONS/TC_seqs.Screening_3_RBPs.csv')], axis=1)
        # creat new order of columns in final results
        newHeader=['Class','Prob1','Prob0','V1','V2']
        result = result[newHeader]
        result = result.sort_values(by=['Prob1'], ascending=False)
        result.to_csv(sFile3[:-4]+'_predictions.csv', index=True)

        print("Time",(time.time() - start)/60,"mins")

In [ ]:
print("==> Chekc the results:")
print(sFile1[:-4]+'_predictions.csv')
print(sFile2[:-4]+'_predictions.csv')
print(sFile3[:-4]+'_predictions.csv')

Hf with ML!@muntisa